# COGS515 - Homework 1

> Name Surname

> Student No

**Don't forget to enter your name and student no above**

In [ ]:
import time
# You may use time module to keep track of the time of your algortihm.

### Problem Formulation

#### State

The initial state of the problem will be represented by a list as follows. Note that ´1´s represent pegs, ´2´s represent empty holes on the board and ´0´s represent inaccessible locations (these are outside the Solo-Test board).

```
[0, 0, 1, 1, 1, 0, 0,
 0, 0, 1, 1, 1, 0, 0,
 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 2, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1,
 0, 0, 1, 1, 1, 0, 0,
 0, 0, 1, 1, 1, 0, 0]
```

Also note that this is a list so it is 1-dimensional, but our actions will consider as if it has 2-dimensions, for example we cannot move a peg in index ´21´ or ´22´ left because that is the left end of the board. So we will need to check valid moves depending on the accessible locations and the edges of the board.

#### Problem Class
Below we define the class of our problem:


In [91]:
up, right, down, left = 0, 1, 2, 3

class SoloTest():
    
    def __init__(self, goal=1):
        # Initial state
        self.initial = [0, 0, 1, 1, 1, 0, 0,
                        0, 0, 1, 1, 1, 0, 0,
                        1, 1, 1, 1, 1, 1, 1,
                        1, 1, 1, 2, 1, 1, 1,
                        1, 1, 1, 1, 1, 1, 1,
                        0, 0, 1, 1, 1, 0, 0,
                        0, 0, 1, 1, 1, 0, 0]
        self.goal = goal
     
    # Goal test - if number of remaining pegs is less than the goal (default goal is 1 peg)
    def goal_test(self,state):
        return state.count(1) <= self.goal

    
    # Check moves if a move is valid
    def checkMove(self, i, state, direction):
        x = i % 7
        y = int(i / 7)
        
        if direction == up:
            if y > 1 and state[i - 7] == 1 and state[i - 14] == 2:
                return True
        elif direction == left:
            if x > 1 and state[i - 1] == 1 and state[i - 2] == 2:
                return True
        elif direction == right:
            if x < 5 and state[i + 1] == 1 and state[i + 2] == 2:
                return True
        elif direction == down:
            if y < 5 and state[i + 7] == 1 and state[i + 14] == 2:
                return True
        return False

            
    # Returns available actions for a state
    def actions(self, state):
        moves = []
        for i in range(len(state)):
            if state[i] == 1:
                if(self.checkMove(i, state, up)):
                    moves.append((i,up))
                if(self.checkMove(i, state, right)):
                    moves.append((i,right))
                if(self.checkMove(i, state, down)):
                    moves.append((i,down))
                if(self.checkMove(i, state, left)):
                    moves.append((i,left))
                else:
                    isolated = True
        return moves
    
    
    # Transition model - returns the resulting state for a state and action
    def result(self, state, action):
        s = list(state)
        location = action[0]
        direction = action[1] # 0 = up 1 = right 2 = down 3 = left
        s[location] = 2
        if direction == up:
            s[location - 7] = 2
            s[location - 14] = 1
        elif direction == right:
            s[location + 1] = 2
            s[location + 2] = 1
        elif direction == down:
            s[location + 7] = 2
            s[location + 14] = 1
        elif direction == left:
            s[location - 1] = 2
            s[location - 2] = 1
        return tuple(s)
    
    # Action cost - 1 per movement. Action cost is irrelevant in this case because there is a fixed number of actions to reach to the goal state
    def action_cost(self, state1, action, state2):
        return 1
    


Below are some functions that you may be useful when implementing your search algorithms
- **Node**: The node data structure for the search tree.
- **expand()**: Function for expanding the nodes in the frontier.
- **boardsolo**: A function to format the state for printing (e.g. print(boardsolo(node.state))

In [92]:
class Node:
    def __init__(self,state,parent=None,action=None,path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        if parent:
            self.depth = parent.depth + 1
        else: 
            self.depth = 0
            
    def path(self):
        ## Path method returns a path from the initial state to the node's state
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

        
            
def expand(problem, node):
    s = node.state
    nodes = []
    for action in problem.actions(s):
        s1 = problem.result(s,action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)


def boardsolo(board, fmt=(7 * '{} {} {} {} {} {} {}\n')):
    return fmt.format(*board).replace('0', '_')



## a) Depth-First Search

Implement the depth-first search algorithm for the problem. Run the algorithm and report **path_cost**, **path_to_goal**, **nodes_expanded**, **max_search_depth**, **max_frontier_nodes** and **running_time** for the solution (check homework for more information). You can implement an additional function to gather these parameters and print them in a readable format.


In [93]:
st_problem = SoloTest()

##Depth First Algorithm
def depth_first(problem):
    maxFrontierNum = 0
    if problem.goal_test(problem.initial):
        node = Node(problem.initial)
        # print("Initial state is the solution!")
        return node, 0, 0
    else:
        node = Node(problem.initial)
        frontier = list(expand(problem, node))
        # Print first expanded and track the expansion step number
        cnt_expanded = 1
        reached = {tuple(problem.initial): node}
        
        while len(frontier) > 0:
            # Keeping track of maximum frontier length
            if maxFrontierNum < len(frontier):
                maxFrontierNum = len(frontier)
            # Stack is used for depth first algorithm
            node = frontier.pop()
            if problem.goal_test(node.state):
               # print("Solved")
                return node, cnt_expanded, maxFrontierNum
            else:
                for child in expand(problem, node):
                    s = child.state
                    # Count expanded nodes
                    cnt_expanded += 1
                    if s not in reached or child.path_cost < reached[s].path_cost:
                        reached[s] = child
                        frontier.append(child)
                    
        print("Could not be solved!")
        return False
            
########################
# Function to print the solution path
def printSolution(result):
    steps = []
    node = result
    steps.append(node.state)
    while(node.parent):
        node = node.parent
        steps.append(node.state)
    steps.reverse()
    num = 0
    for state in steps:
        print("Step",num)
        print("-------------------")
        print(boardsolo(state))
        print("-------------------")
        num += 1
        
# Algorithm run, reports
%time solution = depth_first(st_problem)
print("nodes_expanded: %s" % solution[1])
print("max_frontier_nodes: %s" % solution[2])
print("path_cost: %s" % solution[0].path_cost)
print("max_search_depth: %s" % solution[0].depth)
print("\npath_to_goal:")
printSolution(solution[0])

Wall time: 483 ms
nodes_expanded: 30900
max_frontier_nodes: 117
path_cost: 31
max_search_depth: 31

path_to_goal:
Step 0
-------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 2 1 1 1
1 1 1 1 1 1 1
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _

-------------------
Step 1
-------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 2 1 1 1
_ _ 1 2 1 _ _
_ _ 1 1 1 _ _

-------------------
Step 2
-------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 2 2 1
_ _ 1 2 1 _ _
_ _ 1 1 1 _ _

-------------------
Step 3
-------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 2 1
_ _ 1 2 2 _ _
_ _ 1 1 2 _ _

-------------------
Step 4
-------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 2 1
_ _ 1 2 2 _ _
_ _ 2 2 1 _ _

-------------------
Step 5
-------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 2 2 1 1
_ _ 1 2 2 _ _
_ _ 2 2 1 _ _

-------------------
Step 6
---

## b) Depth-Limited Search

Implement the depth-limited search algorithm for the problem. Run the algorithm and report **path_cost**, **path_to_goal**, **nodes_expanded**, **max_search_depth**, **max_frontier_nodes** and **running_time** for the solution (check homework for more information). You can implement an additional function to gather these parameters and print them in a readable format.


In [ ]:
st_problem = SoloTest()

# Depth Limited Algorithm
def depth_limited(problem, limit):
    maxFrontierNum = 0
    if problem.goal_test(problem.initial):
        print("Initial state is the solution!")
        return True
    else:
        node = Node(problem.initial)
        frontier = list(expand(problem, node))
        cnt_expanded = 1 # Expanded node number
        reached = {tuple(problem.initial): node}
        
        while len(frontier) > 0:
            # Keeping track of maximum frontier length
            if maxFrontierNum < len(frontier):
                maxFrontierNum = len(frontier)
            # Stack is used for depth limited algorithm
            node = frontier.pop()
            if problem.goal_test(node.state):
               # print("Solved")
                return node, cnt_expanded, maxFrontierNum
            elif node.depth < limit:
                for child in expand(problem, node):
                    s = child.state
                    # Count expanded nodes
                    cnt_expanded += 1
                    if s not in reached or child.path_cost < reached[s].path_cost:
                        reached[s] = child
                        frontier.append(child)
                    
        print("Could not be solved!")
        return False
            
########################
# Function to print the solution path
def printSolution(result):
    steps = []
    node = result
    steps.append(node.state)
    while(node.parent):
        node = node.parent
        steps.append(node.state)
    steps.reverse()
    num = 0
    for state in steps:
        print("Step",num)
        print("-------------------")
        print(boardsolo(state))
        print("-------------------")
        num += 1
        
# Algorithm run, reports
%time solution = depth_limited(st_problem, 35)
print("nodes_expanded: %s" % solution[1])
print("max_frontier_nodes: %s" % solution[2])
print("path_cost: %s" % solution[0].path_cost)
print("max_search_depth: %s" % solution[0].depth)
print("\npath_to_goal:")
printSolution(solution[0])

# It finds the same depth as depth_first algorithm. Note that when the algorithm does not reach a solution for
# a given depth limit, the time complexity becomes the same as breadth first search's and it becomes very long to solve. (For example, limit = 10 case)
#########################

## c) Iterative Deepening Depth-Limited Search

Implement the iterative deepening depth-limited search algorithm for the problem. Run the algorithm and report **path_cost**, **path_to_goal**, **nodes_expanded**, **max_search_depth**, **max_frontier_nodes** and **running_time** for the solution (check homework for more information). You can implement an additional function to gather these parameters and print them in a readable format.


In [ ]:
st_problem = SoloTest()
# Depth Limited Algorithm to be used in Iterative Deepening Depth-Limited Search
def depth_limited(problem, limit):
    maxFrontierNum = 0
    if problem.goal_test(problem.initial):
        print("Initial state is the solution!")
        return True
    else:
        node = Node(problem.initial)
        frontier = list(expand(problem, node))
        cnt_expanded = 1 # Expanded node number
        reached = {tuple(problem.initial): node}
        
        while len(frontier) > 0:
            # Keeping track of maximum frontier length
            if maxFrontierNum < len(frontier):
                maxFrontierNum = len(frontier)
            # Stack is used for depth limited algorithm
            node = frontier.pop()
            if problem.goal_test(node.state):
                print("Solved")
                return node, cnt_expanded, maxFrontierNum
            elif node.depth < limit:
                for child in expand(problem, node):
                    s = child.state
                    # Count expanded nodes
                    cnt_expanded += 1
                    if s not in reached or child.path_cost < reached[s].path_cost:
                        reached[s] = child
                        frontier.append(child)
                    
        print("Could not be solved!")
        return False

# Iterative Deepening Depth-Limited Search using Depth-Limited Search
def iter_deep_df(problem, iter_limit):
    for x in range(iter_limit):
        depth_limited(problem, x)

########################
# Function to print the solution path
def printSolution(result):
    steps = []
    node = result
    steps.append(node.state)
    while(node.parent):
        node = node.parent
        steps.append(node.state)
    steps.reverse()
    num = 0
    for state in steps:
        print("Step",num)
        print("-------------------")
        print(boardsolo(state))
        print("-------------------")
        num += 1
        
# Algorithm run, reports
%time solution = iter_deep_df(st_problem, 4)
print("nodes_expanded: %s" % solution[1])
print("max_frontier_nodes: %s" % solution[2])
print("path_cost: %s" % solution[0].path_cost)
print("max_search_depth: %s" % solution[0].depth)
print("\npath_to_goal:")
printSolution(solution[0])

# If algorithm does not reach to a solution, the object "solution" is never created, resulting in an error on reportings.

## d) A*

Implement the A* algorithm for the problem (with a heuristic). Run the algorithm and report **path_cost**, **path_to_goal**, **nodes_expanded**, **max_search_depth**, **max_frontier_nodes** and **running_time** for the solution (check homework for more information). You can implement an additional function to gather these parameters and print them in a readable format.

In [90]:
st_problem = SoloTest()
r_st_problem = r_SoloTest()

def st_h(problem, state):
    temp = problem
    temp.initial = state
    return depth_first(temp)[0].

# Take first element for sort
def takeFirst(elem):
    return elem[0]
    
def a_star(problem, r_problem):
    maxFrontierNum = 0
    if problem.goal_test(problem.initial):
        print("Initial state is the solution!")
        return True
    else:
        node = Node(problem.initial)
        frontier = list(expand(problem, node))
        # Add the heuristic values to frontier nodes
        for h in range(len(frontier)):
            frontier[h] = [st_h(r_problem, node.state), frontier[h]]
        cnt_expanded = 1 # Expanded node number
        reached = {tuple(problem.initial): node}

        while len(frontier) > 0:
            # Sort the frontier according to heuristic value
            frontier.sort(key=takeFirst)
            # Keeping track of maximum frontier length
            if maxFrontierNum < len(frontier):
                maxFrontierNum = len(frontier)
            # Pop the node with smallest heuristic
            node = frontier.pop(0)
            # Take the second list item on popped element (which is a node)
            if problem.goal_test(node[1].state):
                print("Solved")
                return node[1], cnt_expanded, maxFrontierNum
            else:
                for child in expand(problem, node[1]):
                    s = child.state
                    # Count expanded nodes
                    cnt_expanded += 1
                    if s not in reached or child.path_cost < reached[s].path_cost:
                        reached[s] = child
                        frontier.append([st_h(r_problem, s), child])

        print("Could not be solved!")
        return False


########################
# Function to print the solution path
def printSolution(result):
    steps = []
    node = result
    steps.append(node.state)
    while(node.parent):
        node = node.parent
        steps.append(node.state)
    steps.reverse()
    num = 0
    for state in steps:
        print("Step",num)
        print("-------------------")
        print(boardsolo(state))
        print("-------------------")
        num += 1
        
# Algorithm run, reports
%time solution = a_star(st_problem, r_st_problem)
print("nodes_expanded: %s" % solution[1])
print("max_frontier_nodes: %s" % solution[2])
print("path_cost: %s" % solution[0].path_cost)
print("max_search_depth: %s" % solution[0].depth)
print("\npath_to_goal:")
printSolution(solution[0])

KeyboardInterrupt: 

nodes_expanded: 7282
max_frontier_nodes: 105
path_cost: 31
max_search_depth: 31

path_to_goal:
Step 0
-------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 2 1 1 1
1 1 1 1 1 1 1
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _

-------------------
Step 1
-------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 2 1 1 1
_ _ 1 2 1 _ _
_ _ 1 1 1 _ _

-------------------
Step 2
-------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 2 2 1
_ _ 1 2 1 _ _
_ _ 1 1 1 _ _

-------------------
Step 3
-------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 2 1
_ _ 1 2 2 _ _
_ _ 1 1 2 _ _

-------------------
Step 4
-------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 1 1 2 1
_ _ 1 2 2 _ _
_ _ 2 2 1 _ _

-------------------
Step 5
-------------------
_ _ 1 1 1 _ _
_ _ 1 1 1 _ _
1 1 1 1 1 1 1
1 1 1 1 1 1 1
1 1 1 2 2 1 1
_ _ 1 2 2 _ _
_ _ 2 2 1 _ _

-------------------
Step 6
-------------------
_ 

In [ ]:
r_st_problem = r_SoloTest()
solution = depth_first(r_st_problem)
print("nodes_expanded: %s" % solution[1])
print("max_frontier_nodes: %s" % solution[2])
print("path_cost: %s" % solution[0].path_cost)
print("max_search_depth: %s" % solution[0].depth)
print("\npath_to_goal:")
printSolution(solution[0])